#Gated Recurrent Units (GRUs)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('Small.csv', index_col=False)
texts = df['Tweet']
labels = df['Sarcasm'].map({'yes': 1, 'no': 0})

In [3]:
max_len = 100
vocab_size = 10000
embedding_dim = 128

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

padded_sequences = pad_sequences(sequences, maxlen=max_len)

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [4]:
def predict_sarcasm(text, tokenizer, model, max_len):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(padded_sequence)
    if prediction > 0.5:
        return "This text is predicted to be sarcastic."
    else:
        return "This text is predicted to be non-sarcastic."

In [5]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GRU(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(32))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

new_text = "My name is Tanisha."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)

Epoch 1/10
174/174 [==============================] - 35s 170ms/step - loss: 0.3961 - accuracy: 0.8247 - val_loss: 0.2779 - val_accuracy: 0.8961
Epoch 2/10
174/174 [==============================] - 23s 130ms/step - loss: 0.1825 - accuracy: 0.9291 - val_loss: 0.2671 - val_accuracy: 0.8911
Epoch 3/10
174/174 [==============================] - 26s 148ms/step - loss: 0.0831 - accuracy: 0.9733 - val_loss: 0.2723 - val_accuracy: 0.8932
Epoch 4/10
174/174 [==============================] - 24s 139ms/step - loss: 0.0525 - accuracy: 0.9841 - val_loss: 0.3882 - val_accuracy: 0.8968
Epoch 5/10
174/174 [==============================] - 23s 133ms/step - loss: 0.0283 - accuracy: 0.9921 - val_loss: 0.4293 - val_accuracy: 0.8860
Epoch 6/10
174/174 [==============================] - 24s 138ms/step - loss: 0.0211 - accuracy: 0.9948 - val_loss: 0.4298 - val_accuracy: 0.8918
Epoch 7/10
174/174 [==============================] - 24s 139ms/step - loss: 0.0147 - accuracy: 0.9959 - val_loss: 0.4824 - val_ac

In [8]:
from sklearn.metrics import f1_score

In [10]:
y_pred = (model.predict(X_test) > 0.5).astype(int)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Score (weighted): {f1:.4f}")

44/44 [==============================] - 2s 39ms/step
F1 Score (weighted): 0.8982


In [6]:
new_text = "My mom asked me this question as well."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)

1/1 [==============================] - 0s 31ms/step
This text is predicted to be sarcastic.


In [11]:
new_text = "Sure, let's just add this to my already overflowing to-do list."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)

1/1 [==============================] - 0s 49ms/step
This text is predicted to be sarcastic.


In [12]:
new_text = "Sure, let's just add this to my already overflowing to-do list."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)

1/1 [==============================] - 0s 30ms/step
This text is predicted to be sarcastic.
